# Evaluation of the metrics 

This notebook contains the various procedures followed to evaluate the proposed metrics, density & coverage

### Install the dependencies

In [ ]:
# dependencies
import os 
os.environ["GIT_PYTHON_REFRESH"] = "quiet" 
#!module load git
import foolbox as fb
import torch
import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
import numpy as np
from n2gem.metrics import gem_build_coverage, gem_build_density
from n2gem.aux_funcs import gem_build_tree
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from util_models import *

In [ ]:
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from fastai.vision.all import *
from fastai.vision import *

In [ ]:
import medmnist
from medmnist import INFO, Evaluator

In [ ]:
from utils import *

Fix the seed generator

In [ ]:
torch.manual_seed(42) 
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
np.random.seed(42)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#print(device); #print(torch.cuda.memory_allocated())
#torch.cuda.device_count()

In [ ]:
import pymde

------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------

# Datasets

### MNIST dataset 

In [ ]:
def load_mnist():
    """Function to load the mnist
        The following transform is aaplied: Normalize:(0.1307,)(0.3081,)
    
    """
    transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081))
    ])
    train_set = torchvision.datasets.MNIST('./files/', train=True, download=True, transform=transforms)
    test_set = torchvision.datasets.MNIST('./files/', train=False, download=True, transform=transforms)
    
    return train_set, test_set

train_set, test_set = load_mnist()

### MEDMNIST dataset 

In [ ]:
# run this cell to download the medmnist datasets
# change the data_flag accordingly
data_flag = 'pathmnist'
#data_flag = 'organamnist'
#download = True

NUM_EPOCHS = 3
BATCH_SIZE = 128
lr = 0.001

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [ ]:
# load the respective dataset images and labels
npz_file = np.load('/.medmnist/organamnist.npz') # enter the file path here

train_imgs = npz_file['train_images']
test_imgs = npz_file['test_images']
val_imgs = npz_file['val_images']

train_labels = npz_file['train_labels']
test_labels = npz_file['test_labels']
val_labels = npz_file['val_labels']

arr_X_dataset = np.concatenate([train_imgs, test_imgs, val_imgs])
arr_Y_dataset = np.concatenate([train_labels, test_labels, val_labels])

In [ ]:
# the datatransform for the MedMNIST dataset
# uncomment the required line for pathMNIST or OrganMNIST
data_transform = torchvision.transforms.Compose([
    #torchvision.transforms.ToPILImage(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Grayscale(),
    #torchvision.transforms.Normalize((0.74,0.53,0.71), (0.12,0.18,0.13)) # pathmnist RGB normalize values
    #torchvision.transforms.Normalize((0.4657), (0.2936))  # organmnist
    
])

#### form the datset using the ```LoadMed```

In [ ]:
dataset = LoadMed(arr_X_dataset, arr_Y_dataset, data_transform)

-------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------

### Combine the datasets and stratify split 
- Combine train and test
- form the model_dataset[training+ test] and validation set
- obtain training & test set to train on the model from model_dataset

In [ ]:
# mnist
X_trainset, X_testset, X_validation, y_trainset, y_testset, y_validation = create_dataset(False, 0.03, 0.2, train_set=train_set, test_set=test_set)

# pathmnist
#X_trainset, X_testset, X_validation, y_trainset, y_testset, y_validation = create_dataset(dataset, 0.05, 0.2)

# organmnist
#X_trainset, X_testset, X_validation, y_trainset, y_testset, y_validation = create_dataset(dataset, 0.08, 0.2)

### Convert the datasets to Torch.TensorDataset

In [ ]:
train_set, test_set,  model_images, model_labels, validation_set = convert_tensor(X_trainset, X_testset, y_trainset, y_testset, X_validation,  y_validation)

#### Save the model_dataset & validation_set images

In [ ]:
torch.save(model_images, 'model_images.pt')
torch.save(model_labels, 'model_labels.pt')

torch.save(X_validation, 'validation_images.pt')
torch.save(y_validation, 'validation_labels.pt')

#### Dataloader for the CNN models

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=True)

-------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------

### Define the CNN model

- the model architecture for the MedMNIST dataset has been adapted from the MedMNIST repository(https://github.com/MedMNIST/MedMNIST)

Get the parameteres from the model

In [ ]:
model = MnNet() #PathNet #OrgNet
print(sum(p.numel() for p in model.parameters() if p.requires_grad))
#print(sum(p.numel() for p in model1.parameters() if p.requires_grad))

### Fastai classifier
- Fastai takes the available device by default

In [ ]:
data = DataLoaders(train_loader, test_loader)
learn_ = Learner(data, MnNet(), loss_func=F.nll_loss, opt_func=Adam, metrics=[accuracy]) #f1score = F1Score(average='macro')

In [ ]:
learn_.fit_one_cycle(2)

In [ ]:
learn_.unfreeze()
learn_.lr_find()

In [ ]:
learn_.fit_one_cycle(8, lr_max=1e-4)

In [ ]:
learn_.recorder.plot_loss()

In [ ]:
learn_.fine_tune(5)

### - Save the fastai classifier
- By default it is saved in the /models folder with .pth extension

In [ ]:
learn_.save('./fastai_model')

- Load the model(fastai) and save it as torch model for foolbox compatibility

In [ ]:
model_new = learn_.load('fastai_model')
torch.save(model_new.model.state_dict(), 'fastai_model_mnist_weights.pt')

---------------------------------------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------------------------------------

### Load model for the attack

In [ ]:
# load the respective models for the attack
MyModel = MnNet() # MnNet, PathNet, OrgNet

# --> for the attack change model to model.eval

# mnist
MyModel.load_state_dict(torch.load('chkpt_files/fastai_MnNet_weights.pth', map_location=device))

#pathmnist
#MyModel.load_state_dict(torch.load('chkpt_files/fastai_pathmnist_96_weights.pt', map_location=device)) 

#organmnist
#MyModel.load_state_dict(torch.load('chkpt_files/fastai_organmnist_99_weights.pt', map_location=device)) #organmnist

MyModel.eval()
#print(MyModel)

### Create a Pytorch model for foolbox attack

In [ ]:
# mnist
preprocess = dict(mean=0.1307, std=0.3081)

# pathmnist RGB normalize values
#preprocess = dict(mean=[0.74,0.53,0.71], std=[0.12,0.18,0.13], axis=-3)

# organmnist
#preprocess = dict(mean=0.4657, std=0.2936)

bound = (0, 1)
original_model = fb.PyTorchModel(MyModel, bounds=bound, preprocessing=preprocess)

- Attack with 20 values of epsilons

In [ ]:
attack2 = fb.attacks.FGSM()
epsilon = np.linspace(0.0, 1, num=20)

<a id='Attack the model'></a>
## Attack the model

- ```FGSM attack```
- Attack the model using validation_dataset
- Stratify split model_set --> mnist, pathmnist

#### Load the the images

the images can saved in the ```images``` folder and loaded again for the attack. The ```images``` folder is not included in the repo. for space constrictions.

 mnist

In [ ]:
md_images = torch.load('images/model_dataset_images.pt', map_location='cpu').cpu()
md_labels = torch.load('images/model_dataset_labels.pt', map_location='cpu').cpu()

vali_images = torch.load('images/validation_images.pt', map_location='cpu')
vali_labels = torch.load('images/validation_labels.pt', map_location='cpu')

#### For the ```MNIST``` and ```PathMNIST``` dataset, the model dataset was split into 20,000 & 30,000 images respectively

In [ ]:
# split the model_dataset to obtain 20000 images for the attack

# mnist
_, X_images, _, y_labels = train_test_split(md_images.numpy(), md_labels.numpy(), test_size=0.29455, random_state=42, stratify=md_labels.numpy())

# pathmnist
#_, X_images,_, y_labels = train_test_split(md_images.numpy(), md_labels.numpy(), test_size=0.3, random_state=42, stratify=md_labels.numpy())

Reshape and form the model_dataset tensors--> named as images

In [ ]:
# mnist
images = ep.astensor(torch.from_numpy(np.array(X_images)).to(device))
#images.shape

# pathmnist
#images = ep.astensor(torch.from_numpy(np.array(X_images)).to(device)) # 30000 model dataset images
#labels = ep.astensor(torch.from_numpy(np.array(y_labels)).to(device))


# organmnist
#images = ep.astensor(md_images.to(device))

images.shape

### Compute Metrics- Density & Coverage

In [ ]:
# resize the real images
real = images.raw.view(images.shape[0], -1)
real.shape, type(real)

#### Reference density & coverage
-> model_dataset and validation_dataset

In [ ]:
# images from the validation set
gen_validate = torch.from_numpy(np.array(vali_images).reshape(len(vali_images), -1)).to(device)
gen_labels = torch.from_numpy(np.array(vali_labels).reshape(-1)).to(device)
print(gen_validate.shape)


density_validate = gem_build_density(real, real.shape[0], gen_validate, 'indexflatl2')
coverage_validate = gem_build_coverage(real, real.shape[0], gen_validate, 'indexflatl2')
print(f'density: {density_validate:.5f}, coverage: {coverage_validate:.5f}')

#### Attack the model using validation dataset

- convert the validation_images/labels into ep.tensor for foolbox attack compatibility

In [ ]:
vali_imagesx = ep.astensor(torch.from_numpy(np.array(vali_images)).to(device))
vali_labelsx = ep.astensor(torch.from_numpy(np.array(vali_labels).reshape(-1)).to(device))
print(vali_imagesx.shape, vali_labelsx.shape)

### Perform the attack

In [ ]:
#from utils import model_attack
adv_vali, adv_info_vali = model_attack(attack2, original_model, vali_imagesx, vali_labelsx, epsilon)

In [ ]:
acc = 1-adv_info_vali.float32().mean(axis=-1)

#### Compute Density & Coverage 

##### Adversarial metric
compute the metrics between the model_dataset and generated adv.samples(not the feature space)
- model_dataset and validation adversarial samples

In [ ]:
vali_den = []
vali_cov = []

for i in range(len(epsilon)):

    # generated adversarial for each epsilon(convert from eagerpy --> torch and reshape)
    gen = adv_vali[i].raw.view(adv_vali[i].shape[0], -1)

    # density
    vali_den.append(gem_build_density(real, real.shape[0], gen, 'indexflatl2'))

    # coverage
    vali_cov.append(gem_build_coverage(real, real.shape[0], gen, 'indexflatl2'))

In [ ]:
robust_accuracy = []
for i in range(20):
    acc = 1 - adv_info_vali[i, :].raw.cpu().numpy().astype(np.float32).mean(axis=-1)
    robust_accuracy.append(acc)

In [ ]:
print("Density")
print(f"Reference: density: {density_validate:.5f}")
density_data = []

for i in range(len(epsilon)):
    #print("Epsilon: {:.5f}, Accuracy: {:.2f}%, Vali_adv_density: {:.5f}".format(epsilon[i], robust_accuracy[i], vali_den[i]))
    density_data.append([epsilon[i], robust_accuracy[i], vali_den[i].cpu().numpy()])

density_data = np.array(density_data)

#### Plot the variation of density & coverage between the benign samples and the adversarial samples

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14,8))
ax[0].plot(epsilon, vali_den, c='b', label='model_vali_adv')
ax[0].plot(epsilon, np.repeat(density_validate.cpu().numpy(), len(epsilon)), ls='--', c='r', label='model_vali')
ax[0].set_xlabel("Epsilon")
ax[0].set_ylabel("Density")
ax[0].set_title("FGSM attack")
ax[0].legend()
ax[1].plot(epsilon, vali_cov, c='b', label='model_vali_adv')
ax[1].plot(epsilon, np.repeat(coverage_validate.cpu().numpy(), len(epsilon)), ls='--', c='r', label='model_vali')
ax[1].set_xlabel("Epsilon")
ax[1].set_ylabel("Coverage")
ax[1].set_title("FGSM attack")
ax[1].legend()
fig.tight_layout()
#plt.savefig("FGSM_attack_pathmnist_FX_model_vali_den_cov.png")

In [ ]:
print("Coverage")
print(f"Refernce: coverage: {coverage_validate}")
print(f"Model_dataset & adversarial samples")
coverage_data = []
for i in range(len(epsilon)):
    #print("Epsilon: {:.5f}, Accuracy: {:.2f}%, Vali_adv_coverage: {:.5f}".format(epsilon[i], robust_accuracy[i], vali_cov[i]))
    coverage_data.append([epsilon[i], robust_accuracy[i], vali_cov[i].cpu().numpy()])

coverage_data = np.array(coverage_data)

### Save/write the density_data & coverage_data

- info. in each file: 
    - type of attack with model (FX- corresponds to feature extracted computation of metrics)
    - attack model with model_dataset -> model_dataset & adversarial samples
    - Column: Epsilon | Model_accuracy after attack | Metric between model_dataset & adv. samples 
    - Metric between model_dataset and validation_set

In [ ]:
with open(<f_name>, 'w') as newfile:
    newfile.write("# FGSM attack NewNet model with FX model_dataset(30000 images)" + "\n" +
                 "# and validation_images(5329) 20 epsilon" + "\n" +
                 "# Epsilon Model_accuracy FX_Vali_adv_density" + "\n" +
                 "# FX_Model_dataset & FX_validation set: " + str(density_validate.cpu().numpy()) + "\n" )

with open(<f_name>, 'w') as newfile:
    newfile.write("# FGSM attack NewNet model with FX model_dataset(30000 images)" + "\n" +
                 "# and validation_images(5329) 20 epsilon" + "\n" +
                 "# Epsilon Model_accuracy  FX_Vali_adv_coverage" + "\n" +
                 "# FX_Model_dataset & FX_validation set: " + str(coverage_validate.cpu().numpy()) + "\n")
with open(<f_name>, 'a') as addfile:
    np.savetxt(addfile, density_data)
    
with open(<f_name>, 'a') as addfile:
    np.savetxt(addfile, coverage_data)

### Mixture of adv. samples

- the analysis on the mixture of adv. samples into benign samples for FGSM attack
- random mixture of adv. samples into benign samples(validation_set) --> mixture dataset
- proportionate quantities: 25%, 50%, 75%
- compute metrics between the model_dataset and the created mixture dataset

In [ ]:
# enter the no. of validation samples and the split size
xx = np.random.choice(np.arange(<vali_sample_size>), size=int(<split_size>*<vali_sample_size>), replace=False)
mask = np.zeros(2100, dtype=np.bool)
mask[xx] = True

In [ ]:
vali25_den=[]; vali25_cov=[]

for i in range(len(epsilon)):
    # for random mixing
    adv_vali25_imgs = adv_vali[i].raw[mask]
    total_vali_set = torch.cat([vali_imagesx.raw[~mask], adv_vali25_imgs])
    
    gen = total_vali_set.view(total_vali_set.shape[0], -1)
    
    # density
    vali25_den.append(gem_build_density(real, real.shape[0], gen, 'indexflatl2'))
    
    # coverage
    vali25_cov.append(gem_build_coverage(real, real.shape[0], gen, 'indexflatl2'))

In [ ]:
metrics25 = []
for i in range(len(epsilon)):
    metrics25.append([epsilon[i], robust_accuracy[i], vali25_den[i].cpu().numpy(), vali25_cov[i].cpu().numpy()])
    
metrics25 = np.array(metrics25)

In [ ]:
with open('FGSM_attack_mnist_NewNet_50randmix_valiadv.dat', 'w') as newfile:
    newfile.write("# FGSM attack NewNet model with model_dataset(20000 images)" + "\n" +
                 "# and validation_images(2100) 20 epsilon" + "\n" +
                 "# 50% mix of adv vali samples random" + "\n" +
                 "# Epsilon model_acc Vali25_adv_den Vali25_adv_cov" + "\n" )
                 
with open('FGSM_attack_mnist_NewNet_50randmix_valiadv.dat', 'a') as addfile:
    np.savetxt(addfile, metrics25)

------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------

# ```Boundary attack``` on the model

This attack works fine i.e, adversarial samples are created if epsilon is assigned as None

### Attack the model using entire validation_dataset

- the Boundary Attack cannot be implemented if the starting images are not adversaries
- to create these adversarial samples, ```init_attack``` needs to be specified
- any of the attacks inherited from ```Minimization Attack``` can be used for this purpose
- all the initial samples should be strictly adversarial samples

##### Two different attacks were experimented
- ```SaltandPepper```, ```LinearSearchBlendedUniformNoiseAttack```
- modify the hyperparameters within these attacks to generate the adversarial samples
- the generated samples are given as the starting points for the ```BoundaryAttack```
- time complexity increases with changing these hyperparameters

In [ ]:
# load the images from the Attack the model section

vali_attk_images = ep.astensor(torch.from_numpy(np.array(vali_images)).to(device))
vali_attk_labels = ep.astensor(torch.from_numpy(np.array(vali_labels)).to(device))

In [ ]:
vali_attk_images.shape, type(vali_attk_labels)

In [ ]:
BdyAttack = fb.attacks.BoundaryAttack()

### Perform init_attack

- attack using ```s_attack``` or ```n_attack``` with the validation_set 
- generate adv. samples --> ```adv_lsbu```

In [ ]:
s_attack = fb.attacks.SaltAndPepperNoiseAttack(steps=5000, across_channels=True)
n_attack = fb.attacks.LinearSearchBlendedUniformNoiseAttack(steps=2000, directions=3000)
_, adv_lsbu, adv_bdy_info = n_attack(original_model, vali_attk_images, vali_attk_labels.reshape(-1), epsilons=None)

In [ ]:
adv_bdy_info.float32()

In [ ]:
print(f"acc: {1 - adv_bdy_info.float32().mean(axis=-1)}")

### Perform the Boundary attack

- use the generated adv. images as ```starting_pooint``` for the attack

In [ ]:
_, adv_bdy, adv_bdy_info = BdyAttack(original_model, vali_attk_images, vali_attk_labels, epsilons=None) #starting_points=adv_lsbu,

### Compute Density & Coverage

Reference metric

In [ ]:
vali_attk_images.shape, vali_attk_labels.shape, md_images.shape#, real.shape

In [ ]:
#real = md_images.view(md_images.shape[0], -1).to(device)
# images from the validation set
# for pathmnist 30000 model_dataset images are used 
gen_validate = vali_attk_images.raw.view(vali_attk_images.shape[0], -1)
real = images.raw.view(images.shape[0], -1).to(device)

density_validate = gem_build_density(real, real.shape[0], gen_validate, 'indexflatl2')
coverage_validate = gem_build_coverage(real, real.shape[0], gen_validate, 'indexflatl2')
print(density_validate, coverage_validate)

Adversarial metric
model_dataset & validation_dataset adversarials

In [ ]:
gen_adv_val = adv_bdy.raw.view(adv_bdy.shape[0], -1)

model_density_val = gem_build_density(real, real.shape[0], gen_adv_val, 'indexflatl2')
model_coverage_val = gem_build_coverage(real, real.shape[0], gen_adv_val, 'indexflatl2')

print(model_density_val, model_coverage_val)

In [ ]:
print("Density")
print(f"Reference: density: {density_validate:.5f}")
print(f"Adv: density: {model_density_val:.5f}")

### Mixture of adv. samples

- random mixture of adv. samples into validation_set
- compute metrics between mixture_set and model_dataset

In [ ]:
vali25_den=[]; vali25_cov=[]
xx = np.random.choice(np.arange(<vali_sample_size>), size=int(<mix_size>*<vali_sample_size>), replace=False)
#yy = np.random.choice(np.arange(4708), size=int(0.75*4708), replace=False)
mask = np.zeros(<vali_sample_size>, dtype=np.bool)
mask[xx] = True

# for random mixing
adv_vali25_imgs = adv_bdy.raw[mask]
total_vali_set = torch.cat([vali_attk_images.raw[~mask], adv_vali25_imgs])

gen = total_vali_set.view(total_vali_set.shape[0], -1)

# density
vali25_den = gem_build_density(real, real.shape[0], gen, 'indexflatl2')

# coverage
vali25_cov= gem_build_coverage(real, real.shape[0], gen, 'indexflatl2')
print(vali25_den, vali25_cov)